# Python Depedencies

Install the required dependencies as they are described in the requirements.txt file

In [1]:
%pip install -r requirements.txt

  Using cached azure_core-1.29.7-py3-none-any.whl.metadata (36 kB)
  Using cached azure_identity-1.15.0-py3-none-any.whl.metadata (75 kB)
  Using cached azure_search_documents-11.4.0-py3-none-any.whl.metadata (22 kB)
  Using cached semantic_kernel-0.4.5.dev0-py3-none-any.whl.metadata (2.1 kB)
  Using cached openai-1.9.0-py3-none-any.whl.metadata (18 kB)
  Using cached langchain-0.1.6-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_openai-0.0.3-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_community-0.0.19-py3-none-any.whl.metadata (7.9 kB)
  Using cached langchain_core-0.1.23-py3-none-any.whl.metadata (6.0 kB)
  Using cached tiktoken-0.5.2-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached unstructured-0.12.2-py3-none-any.whl.metadata (26 kB)
  Using cached Markdown-3.5.2-py3-none-any.whl.metadata (7.0 kB)
  Using cached

# Use OpenAI API

Before we call any API, we need to retrieve the required values from the `.env` file.

In [42]:
import json     # Import the json module to work with JSON data
import requests # Import the requests module to send HTTP requests
import os       # Import the os module to work with the operating system

# Import the load_dotenv function from the dotenv module to load environment variables from a .env file
from dotenv import load_dotenv

# Load environment variables from a .env file. If successful, print the Azure OpenAI API endpoint
if load_dotenv():
    print("Azure OpenAI API Base: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    # If the .env file could not be loaded, print a message indicating that the Azure OpenAI API endpoint was not found
    print("Azure OpenAI API Endpoint not found.")
    
# Get the values of the environment variables
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")


# url = AZURE_OPENAI_ENDPOINT + "/openai/deployments/" + AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME + "/completions?api-version=" + AZURE_OPENAI_API_VERSION
url = AZURE_OPENAI_ENDPOINT + "/openai/deployments/" + AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME + "/chat/completions?api-version=" + AZURE_OPENAI_API_VERSION

print(f'Remote OpenAI API completions URL: {url}')


Azure OpenAI API Base: https://cog-ilecskbcfhj6i.openai.azure.com/
Remote OpenAI API completions URL: https://cog-ilecskbcfhj6i.openai.azure.com//openai/deployments/turbochatmodel/chat/completions?api-version=2023-09-01-preview


Since we constructed the required URL that we will use, let's call it. We need to pass the API key in the HTTP header

In [44]:
# r = requests.post(url, headers={"api-key": AZURE_OPENAI_API_KEY}, json={"max_tokens": 40, "prompt": "tell me a joke", "temperature": 1.0})
r = requests.post(url, headers={"api-key": AZURE_OPENAI_API_KEY}, json={"messages":[{"role": "user", "content": "tell me a quick joke"}]})


print(json.dumps(r.json(), indent=2))


{
  "id": "chatcmpl-8wnfkbSJHs2lqruAO3zSZ15Pe5gzB",
  "object": "chat.completion",
  "created": 1709024656,
  "model": "gpt-35-turbo",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Why don't scientists trust atoms? Because they make up everything!"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
      

# Use OpenAI API Python Library
In the next snippet we are going to issue the same request but without using the http request directly. Instead we will use the Python `openai` library, which as we will see, simplifies somehow our code. 

In [58]:
# Import the AzureOpenAI class from the openai module
from openai import AzureOpenAI

# Create an instance of the AzureOpenAI class, passing in the Azure endpoint, API key, and API version from environment variables
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_API_VERSION")
)

# Call the chat.completions.create method on the client object, passing in the model name from an environment variable and a list of messages
response = client.chat.completions.create(
    model = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
    messages = [{"role" : "assistant", "content" : "Tell me a quick joke"}],
)

# Print the response from the API call
# print(response)

print('Response: ' + response.choices[0].message.content)
print(response.usage)


Response: Why did the tomato turn red? Because it saw the salad dressing!
CompletionUsage(completion_tokens=14, prompt_tokens=13, total_tokens=27)


# Use Langchain

Langchain simplifies the process of building applications powered by large language models. It supports **Python** and Javascript / Typescript, but in this post, we focus on Python.

In [3]:
from langchain.llms import AzureOpenAI
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage

import os
import openai
from dotenv import load_dotenv

if load_dotenv():
    print("Found Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("No file .env found")

## Create an instance of Azure OpenAI
llm = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)

print("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME: " + os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"))


# Define the prompt 
msg = HumanMessage(content="Tell me short story?")

# Call the API
r = llm.invoke([msg])

# Print the response
print(r.content)

Found Azure OpenAI Endpoint: https://cog-ilecskbcfhj6i.openai.azure.com/
AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME: turbochatmodel
Once upon a time, there was a little girl named Lily who lived in a small village. She loved to spend her days exploring the nearby forest, where she would often encounter all sorts of animals, from squirrels to rabbits to deer.

One day, while wandering through the forest, Lily heard a strange noise coming from a nearby bush. Curious, she cautiously approached the bush and peeked inside. To her surprise, she found a tiny, injured bird lying on the ground.

Feeling sorry for the poor bird, Lily decided to take it home and nurse it back to health. She spent the next few days caring for the bird, feeding it, and keeping it warm until it was strong enough to fly away on its own.

As the days passed, Lily continued to explore the forest, but now she did so with a renewed sense of wonder and appreciation for all the creatures that lived there. And whenever she enc